# BLIP-2 Inference on VQA-RAD Dataset
This notebook loads questions and images from the VQA-RAD dataset and runs inference using the BLIP-2 model (Salesforce/blip2-opt-2.7b).

In [2]:

from transformers import Blip2Processor, Blip2ForConditionalGeneration
import torch
from PIL import Image
import os
import json

device = "cuda" if torch.cuda.is_available() else "cpu"
processor = Blip2Processor.from_pretrained("Salesforce/blip2-opt-2.7b")
model = Blip2ForConditionalGeneration.from_pretrained("Salesforce/blip2-opt-2.7b").to(device)


Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [4]:
import zipfile

zip_path = "/content/VQA_RAD_Image_Folder.zip"  # change name if needed
extract_path = "/content/VQA_RAD Image Folder"

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

image_dir = "/content/VQA_RAD Image Folder"  # or Drive path

In [5]:

# Load your dataset
with open("/content/VQA_RAD Image Folder/VQA_RAD Dataset Public.json", "r") as f:
    data = json.load(f)

# Path to your image folder
image_dir = "/content/VQA_RAD Image Folder/VQA_RAD Image Folder"


In [6]:
# Show a few image names from JSON
print("From dataset:", [sample["image_name"] for sample in data[:5]])

# Show actual image files in the folder
print("From folder:", os.listdir(image_dir)[:5])

From dataset: ['synpic54610.jpg', 'synpic29265.jpg', 'synpic29265.jpg', 'synpic28602.jpg', 'synpic29265.jpg']
From folder: ['synpic56422.jpg', 'synpic34713.jpg', 'synpic676.jpg', 'synpic42951.jpg', 'synpic56799.jpg']


In [11]:

from tqdm import tqdm

results = []
match_count = 0

for sample in tqdm(data[:50]):  # You can increase this number if Colab supports more
    image_path = os.path.join(image_dir, sample["image_name"])
    question = sample["question"]
    answer = sample["answer"]

    if not os.path.exists(image_path):
        continue

    image = Image.open(image_path).convert("RGB")
    inputs = processor(images=image, text=question, return_tensors="pt").to(device)
    generated_ids = model.generate(**inputs)
    generated_text = processor.batch_decode(generated_ids, skip_special_tokens=True)[0].strip()

    results.append({
        "image": sample["image_name"],
        "question": question,
        "ground_truth": answer,
        "predicted": generated_text
    })

    if generated_text.lower() == answer.lower():
        match_count += 1

print(f"Exact Match Accuracy: {match_count} / {len(results)} = {match_count/len(results):.2%}")


100%|██████████| 50/50 [1:42:15<00:00, 122.71s/it]

Exact Match Accuracy: 0 / 50 = 0.00%


In [12]:

import pandas as pd

df = pd.DataFrame(results)
df.head(10)  # Show a sample of the predictions


,image,question,ground_truth,predicted
0,synpic54610.jpg,Are regions of the brain infarcted?,Yes,Are regions of the brain infarcted?
1,synpic29265.jpg,Are the lungs normal appearing?,No,Are the lungs normal appearing?
2,synpic29265.jpg,Is there evidence of a pneumothorax,No,Is there evidence of a pneumothorax?
3,synpic28602.jpg,What type of imaging does this not represent?,ultrasound,What type of imaging does this not represent?
4,synpic29265.jpg,Is this a MRI of the chest?,no,Is this a MRI of the chest?
5,synpic28602.jpg,What is not pictured in this image?,The extremities,What is not pictured in this image?
6,synpic28602.jpg,Is the trachea midline?,yes,Is the trachea midline?
7,synpic28602.jpg,Is there evidence of an aortic aneurysm?,No,Is there evidence of an aortic aneurysm?
8,synpic39460.jpg,Where is the abnormality?,left temporal lobe,Where is the abnormality?
9,synpic28602.jpg,Is there blunting of the costovertebral angles?,No,Is there blunting of the costovertebral angles?


### Attempted Fine-Tuning (Ablation)

Due to resource limitations in Colab (crashing during memory allocation), we attempted but were unable to fine-tune BLIP-2 on the VQA-RAD dataset.

To simulate a minimal extension, we prepared a code block that selectively unfreezes the language modeling head and performs training on a small subset of samples (10–20 pairs). However, even this restricted fine-tuning caused out-of-memory crashes due to the size of the BLIP-2 model (2.7B parameters).

In a future extension, we recommend:

- Using a smaller model (e.g., BLIP-2 with Vicuna-1.5B or a DistilGPT backbone)
- Running the fine-tuning on a T4 or A100 instance (e.g., via Kaggle, Colab Pro, or AWS)
- Alternatively, replacing BLIP-2 with MedViLL or VLT from the original paper

We include our prepared fine-tuning code below for completeness.

In [ ]:
from torch.optim import AdamW
from torch.nn import functional as F
import gc

# Freeze everything but the language modeling head
for param in model.parameters():
    param.requires_grad = False
for param in model.language_model.parameters():
    param.requires_grad = True

optimizer = AdamW(filter(lambda p: p.requires_grad, model.parameters()), lr=1e-5)

model.train()
subset = data[:10]  # keep it small

for i, sample in enumerate(subset):
    try:
        image_path = os.path.join(image_dir, sample["image_name"])
        if not os.path.exists(image_path): continue

        image = Image.open(image_path).convert("RGB")
        question = sample["question"]
        answer = sample["answer"]

        inputs = processor(images=image, text=question, return_tensors="pt").to(device)
        labels = processor.tokenizer(answer, return_tensors="pt").input_ids.to(device)

        outputs = model(**inputs, labels=labels)
        loss = outputs.loss

        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        print(f"Step {i+1}/{len(subset)} - Loss: {loss.item():.4f}")

        # Manual cleanup
        del inputs, labels, outputs, image
        torch.cuda.empty_cache()
        gc.collect()

    except Exception as e:
        print(f"Error on step {i+1}: {e}")

